In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as pl

In [3]:
import mailbox

In [5]:
mboxfile = r"C:\Users\bincy\Downloads\takeout-20240829T144231Z-001\Takeout\Mail\All mail Including Spam and Trash.mbox"
mbox = mailbox.mbox(mboxfile)
mbox

In [6]:
for key in mbox[0].keys():
    print(key)

X-GM-THRID
X-Gmail-Labels
Delivered-To
Received
X-Received
ARC-Seal
ARC-Message-Signature
ARC-Authentication-Results
Return-Path
Received
Received-SPF
Authentication-Results
DKIM-Signature
X-Google-DKIM-Signature
X-Gm-Message-State
X-Google-Smtp-Source
MIME-Version
X-Received
Date
X-Notifications
X-Notifications-Bounce-Info
Message-ID
Subject
From
To
Content-Type


In [7]:
import csv

In [8]:
with open('mailbox.csv','w') as outputfile:
    writer = csv.writer(outputfile)
    writer.writerow(['subject','from','date','to','label','thread'])
    for message in mbox:
        writer.writerow([
            message['subject'],
            message['from'],
            message['date'],
            message['to'],
            message['X-Gmail-Labels'],
            message['X-GM-THRID']
        ]
            
        )

In [9]:
dfs = pd.read_csv('mailbox.csv',names=['subject','from','date','to','label','thread'])

In [10]:
dfs.dtypes

subject    object
from       object
date       object
to         object
label      object
thread     object
dtype: object

In [11]:
dfs['date'] = dfs['date'].apply(lambda x: pd.to_datetime(x, errors='coerce',utc=True))

In [12]:
dfs.dtypes

subject                 object
from                    object
date       datetime64[ns, UTC]
to                      object
label                   object
thread                  object
dtype: object

In [14]:
dfs = dfs[dfs['date'].notna()]

In [15]:
dfs.to_csv('gmail.csv')

In [16]:
dfs.info()

<class 'pandas.core.frame.DataFrame'>
Index: 155 entries, 1 to 155
Data columns (total 6 columns):
 #   Column   Non-Null Count  Dtype              
---  ------   --------------  -----              
 0   subject  154 non-null    object             
 1   from     155 non-null    object             
 2   date     155 non-null    datetime64[ns, UTC]
 3   to       154 non-null    object             
 4   label    155 non-null    object             
 5   thread   155 non-null    object             
dtypes: datetime64[ns, UTC](1), object(5)
memory usage: 8.5+ KB


In [17]:
dfs.head(10)

,subject,from,date,to,label,thread
1,"New material: ""Predicate logics""","""Abha Sharma (Classroom)"" <no-reply@classroom....",2024-08-28 05:32:48+00:00,bincy.24mas10004@vitbhopal.ac.in,"Inbox,Important,Opened,Category Updates",1808608179451448758
2,Your Custom Reading Picks Are Ready.,GeeksforGeeks <no-reply@geeksforgeeks.org>,2024-08-15 05:04:03+00:00,bincy.24mas10004@vitbhopal.ac.in,"Inbox,Important,Opened,Category Updates",1807428609746621869
3,23 MAS Batch part time & 24 MAS Batch full tim...,"""Programme Chair - M. Tech AI and DS"" <pc.mas@...",2024-08-12 08:35:40+00:00,"swagatkumarsamantaray@vitbhopal.ac.in, saravan...","Inbox,Important,Opened,Category Forums",1807170145847332186
4,"Fwd: Reminder: City, University of London Stud...",Assistant Director International Relations VIT...,2024-08-14 08:50:17+00:00,"allstudents@vitbhopal.ac.in, allfaculty@vitbho...","Inbox,Important,Opened,Category Forums",1807352268993026168
5,Generative AI: The Career Move You Need,"""Analytics Vidhya"" <info@newsletters.analytics...",2024-08-11 05:23:30+00:00,bincy.24mas10004@vitbhopal.ac.in,"Inbox,Important,Opened,Category Updates",1807067449487018886
6,FFCS Course Registration (2024_Freshers)_Certa...,FFCS and Timetable Coordinator <ffcs.timetable...,2024-08-15 19:14:34+00:00,allstudents2024@vitbhopal.ac.in,"Inbox,Important,Opened,Category Forums",1807482162524875877
7,PAT Webinar,Assistant Dean First year <ad.fy@vitbhopal.ac.in>,2024-08-11 10:21:10+00:00,allstudents2024@vitbhopal.ac.in,"Inbox,Important,Opened,Category Forums",1807086198920830988
8,Your New IEEE Account,IEEE Confirmation <hello@em3009.ieee.org>,2024-08-08 18:26:46+00:00,bincy.24mas10004@vitbhopal.ac.in,"Inbox,Important,Opened,Category Updates",1806844933354978473
9,=?UTF-8?Q?New_announcement=3A_=22Please_carry_...,"""Abhishek Kumar Shukla (Classroom)"" <no-reply@...",2024-08-06 08:17:58+00:00,bincy.24mas10004@vitbhopal.ac.in,"Inbox,Important,Opened,Category Updates",1806625437288831950
10,"=?UTF-8?Q?Haven=E2=80=99t_decided_yet,_BINCY=3F?=",IEEE Membership <ieee-membership@deliver.ieee....,2024-08-26 20:01:09+00:00,bincy.24mas10004@vitbhopal.ac.in,"Inbox,Important,Opened,Category Updates",1808481630188248383


In [18]:
import re

In [19]:
def extract_email_ID(string):
    email = re.findall(r'<(.+?)>',string)
    if not email:
        email = list(filter(lambda y: '@' in y, string.split()))
    return email[0] if email else np.nan

In [20]:
dfs['from'] = dfs['from'].apply(lambda x: extract_email_ID(x))

In [21]:
myemail = 'bincy.24mas10004@vitbhopal.ac.in'
dfs['label'] = dfs['from'].apply(lambda x: 'sent' if x==myemail else 'inbox')

In [22]:
dfs.drop(columns='to', inplace=True)

In [23]:
dfs.head(10)

,subject,from,date,label,thread
1,"New material: ""Predicate logics""",no-reply@classroom.google.com,2024-08-28 05:32:48+00:00,inbox,1808608179451448758
2,Your Custom Reading Picks Are Ready.,no-reply@geeksforgeeks.org,2024-08-15 05:04:03+00:00,inbox,1807428609746621869
3,23 MAS Batch part time & 24 MAS Batch full tim...,pc.mas@vitbhopal.ac.in,2024-08-12 08:35:40+00:00,inbox,1807170145847332186
4,"Fwd: Reminder: City, University of London Stud...",ad.ir2@vitbhopal.ac.in,2024-08-14 08:50:17+00:00,inbox,1807352268993026168
5,Generative AI: The Career Move You Need,info@newsletters.analyticsvidhya.com,2024-08-11 05:23:30+00:00,inbox,1807067449487018886
6,FFCS Course Registration (2024_Freshers)_Certa...,ffcs.timetable@vitbhopal.ac.in,2024-08-15 19:14:34+00:00,inbox,1807482162524875877
7,PAT Webinar,ad.fy@vitbhopal.ac.in,2024-08-11 10:21:10+00:00,inbox,1807086198920830988
8,Your New IEEE Account,hello@em3009.ieee.org,2024-08-08 18:26:46+00:00,inbox,1806844933354978473
9,=?UTF-8?Q?New_announcement=3A_=22Please_carry_...,no-reply@classroom.google.com,2024-08-06 08:17:58+00:00,inbox,1806625437288831950
10,"=?UTF-8?Q?Haven=E2=80=99t_decided_yet,_BINCY=3F?=",ieee-membership@deliver.ieee.org,2024-08-26 20:01:09+00:00,inbox,1808481630188248383


In [24]:
import datetime
import pytz
def refactor_timezone(x):
    est = pytz.timezone('US/Eastern')
    return x.astimezone(est)

In [25]:
dfs['date'] = dfs['date'].apply(lambda x: refactor_timezone(x))

In [27]:
dfs['dayofweek'] = dfs['date'].apply(lambda x: x.day_name)
dfs['dayofweek'] = pd.Categorical(dfs['dayofweek'], categories = [
    'Monday','Tuesday','Wednesday','Thursday','Friday','Saturday','Sunday'], ordered=True)

In [28]:
dfs['timeofday'] = dfs['date'].apply(lambda x: x.hour + x.minute/60
+ x.second/3600)

In [29]:
dfs['hour'] = dfs['date'].apply(lambda x: x.hour)

In [31]:
dfs['year_int'] = dfs['date'].apply(lambda x: x.year)

In [32]:
dfs['year'] = dfs['date'].apply(lambda x: x.year +
x.dayofyear/365.25)

In [33]:
dfs.index = dfs['date']
del dfs['date']

In [34]:
print(dfs.index.min().strftime('%a, %d %b %Y %I:%M %p'))
print(dfs.index.max().strftime('%a, %d %b %Y %I:%M %p'))
print(dfs['label'].value_counts())

Mon, 24 Jun 2024 06:12 AM
Thu, 29 Aug 2024 06:40 AM
label
inbox    141
sent      14
Name: count, dtype: int64
